In [1]:
import pandas as pd
import numpy as np

In [2]:
# Create single dataset
df = pd.read_csv("../data/athletes_1_20k.txt", sep = "\t")
paths = ["../data/athletes_20k_40k.txt", "../data/athletes_40k_60k.txt", "../data/athletes_60k_80k.txt", 
        "../data/athletes_80k_100k.txt", "../data/athletes_100k_120k.txt", "../data/athletes_120k_140k.txt"]
for p in paths:
    t = pd.read_csv(p, sep = "\t")
    df = df.append(t, ignore_index=True)

In [3]:
"""
# Checks for athletes with the same name
for n in df["Name"].unique():
    t = df.loc[df['Name'] == n]
    sex_count = len(t['Sex'].unique())
    birth_count = len(t['Birth'].unique())
    if sex_count != 1 and birth_count != 1:
        print(n)
        print(sex_count)
        print(birth_count)
"""

'\n# Checks for athletes with the same name\nfor n in df["Name"].unique():\n    t = df.loc[df[\'Name\'] == n]\n    sex_count = len(t[\'Sex\'].unique())\n    birth_count = len(t[\'Birth\'].unique())\n    if sex_count != 1 and birth_count != 1:\n        print(n)\n        print(sex_count)\n        print(birth_count)\n'

In [5]:
# Keep if Status is 'Olympic'
df = df.loc[df["Status"] == "Olympic"]
# Create column for Games year
df["Games_Year"] = df["Games"].apply(lambda x: int(x.split()[0]))
# Create column for Games season (1956 Equestrian Olympics is Summer)
df["is_summer"] = df["Games"].apply(lambda x: 0 if x.split()[1] == "Winter" else 1)
# Create column for athlete age during games
df["Age"] = df["Games_Year"]-df["Birth"]
# Create column for being male
df["Male"] = df["Sex"].apply(lambda x: 1 if x == "Male" else 0)
# Create column for is team event
has_team = df.loc[~df["Team"].isnull()]
has_team_set = set(has_team.groupby(["Sport", "Event"]).groups.keys())
df_se_tups = zip(df["Sport"], df["Event"])
is_team_event = []
for t in df_se_tups:
    set_flag = False
    for e in has_team_set:
        if (t[0] == e[0]) and (t[1] in e[1]):
            is_team_event.append(1)
            set_flag = True
            break
    if not set_flag:
        is_team_event.append(0)
df["is_team_event"] = is_team_event

In [7]:
df.loc[(df["Medal"] == "Gold") & (~df["Pos"].isin(["1", "=1"]) & (df.is_team_event == 0))]
# DQ medal promotion but original winner also reinstated: 151468, 156238

,Name,Sex,Birth,Games,Sport,Event,Status,Team,Pos,Medal,Nat,Games_Year,is_summer,Age,Male,is_team_event
151468,Ferdinand Bie,Male,1888.0,1912 Summer Olympics,Athletics,"Pentathlon, Men",Olympic,NaN,2,Gold,Norway,1912,1,24.0,1,0
156238,Hugo Wieslander,Male,1889.0,1912 Summer Olympics,Athletics,"Decathlon, Men",Olympic,NaN,2,Gold,Sweden,1912,1,23.0,1,0


In [8]:
df.loc[(~df["Medal"].isin(["Gold", "Silver", "Bronze"])) & (df["Pos"].isin(["1", "=1"])) & (df.is_team_event == 0)]

,Name,Sex,Birth,Games,Sport,Event,Status,Team,Pos,Medal,Nat,Games_Year,is_summer,Age,Male,is_team_event


In [9]:
df.loc[(df["Medal"] == "Silver") & (~df["Pos"].isin(["2", "=2"])) & (df.is_team_event == 0)]
# DQ medal promotion but original winner also reinstated: 155810, 159732
# DQ place promotion only: 144614, 254432, 262100

,Name,Sex,Birth,Games,Sport,Event,Status,Team,Pos,Medal,Nat,Games_Year,is_summer,Age,Male,is_team_event
144614,Aikaterini Thanou,Female,1975.0,2000 Summer Olympics,Athletics,"100 metres, Women",Olympic,NaN,1,Silver,Greece,2000,1,25.0,0,0
155810,Charles Lomberg,Male,1886.0,1912 Summer Olympics,Athletics,"Decathlon, Men",Olympic,NaN,3,Silver,Sweden,1912,1,26.0,1,0
159732,Jim Donahue,Male,1885.0,1912 Summer Olympics,Athletics,"Pentathlon, Men",Olympic,NaN,3,Silver,United States,1912,1,27.0,1,0
254432,Navab Nassirshalal,Male,1989.0,2012 Summer Olympics,Weightlifting,"Heavyweight, Men",Olympic,NaN,1,Silver,Islamic Republic of Iran,2012,1,23.0,1,0
262100,Erik Kynard,Male,1991.0,2012 Summer Olympics,Athletics,"High Jump, Men",Olympic,NaN,1,Silver,United States,2012,1,21.0,1,0


In [10]:
df.loc[(~df["Medal"].isin(["Gold", "Silver", "Bronze"])) & (df["Pos"].isin(["2", "=2"])) & (df.is_team_event == 0)]
# DQ place promotion only: 184315

,Name,Sex,Birth,Games,Sport,Event,Status,Team,Pos,Medal,Nat,Games_Year,is_summer,Age,Male,is_team_event
184315,Tatyana Tomashova,Female,1975.0,2012 Summer Olympics,Athletics,"1,500 metres, Women",Olympic,NaN,2,NaN,Russian Federation,2012,1,37.0,0,0


In [11]:
df.loc[(df["Medal"] == "Bronze") & (~df["Pos"].isin(["3", "=3"])) & (df.is_team_event == 0)]
# DQ medal promotion but original winner also reinstated: 135627, 155575
# DQ place promotion only: 243520, 252178, 255990, 258118, 259154
# DQ medal promotion and 2 place promotions: 222139, 223673
# Medal randomly assigned: 120698
# Medal misassigned: 158951
# Out of date: 241210

,Name,Sex,Birth,Games,Sport,Event,Status,Team,Pos,Medal,Nat,Games_Year,is_summer,Age,Male,is_team_event
120698,Albert Pettersson,Male,1885.0,1920 Summer Olympics,Weightlifting,"Middleweight, Men",Olympic,NaN,=2,Bronze,Sweden,1920,1,35.0,1,0
135627,Frank Lukeman,Male,1885.0,1912 Summer Olympics,Athletics,"Pentathlon, Men",Olympic,NaN,4,Bronze,Canada,1912,1,27.0,1,0
155575,Gösta Holmér,Male,1891.0,1912 Summer Olympics,Athletics,"Decathlon, Men",Olympic,NaN,4,Bronze,Sweden,1912,1,21.0,1,0
158951,Mickey Patterson,Female,1926.0,1948 Summer Olympics,Athletics,"200 metres, Women",Olympic,NaN,4,Bronze,United States,1948,1,22.0,0,0
222139,Eşref Apak,Male,1982.0,2004 Summer Olympics,Athletics,"Hammer Throw, Men",Olympic,NaN,2,Bronze,Turkey,2004,1,22.0,1,0
223673,Christina Obergföll,Female,1981.0,2008 Summer Olympics,Athletics,"Javelin Throw, Women",Olympic,NaN,2,Bronze,Germany,2008,1,27.0,0,0
241210,Blessing Okagbare,Female,1988.0,2008 Summer Olympics,Athletics,"Long Jump, Women",Olympic,NaN,2,Bronze,Nigeria,2008,1,20.0,0,0
243520,Bartłomiej Bonk,Male,1984.0,2012 Summer Olympics,Weightlifting,"Heavyweight, Men",Olympic,NaN,2,Bronze,Poland,2012,1,28.0,1,0
252178,Ivan Shtyl,Male,1986.0,2012 Summer Olympics,Canoe Sprint (Canoeing),"Canadian Singles, 200 metres, Men",Olympic,NaN,2,Bronze,Russian Federation,2012,1,26.0,1,0
255990,Derek Drouin,Male,1990.0,2012 Summer Olympics,Athletics,"High Jump, Men",Olympic,NaN,=2,Bronze,Canada,2012,1,22.0,1,0


In [12]:
df.loc[(~df["Medal"].isin(["Gold", "Silver", "Bronze"])) & (df["Pos"].isin(["3", "=3"])) & (df.is_team_event == 0)]
# DQ place promotion only: 219012, 222155, 223700, 224842, 251912, 253584, 260145
# Medal misassigned: 132330
# Out of date: 241189

,Name,Sex,Birth,Games,Sport,Event,Status,Team,Pos,Medal,Nat,Games_Year,is_summer,Age,Male,is_team_event
132330,Shirley Strickland de la Hunty,Female,1925.0,1948 Summer Olympics,Athletics,"200 metres, Women",Olympic,NaN,3,NaN,Australia as Shirley Strickland,1948,1,23.0,0,0
219012,Nadezhda Ostapchuk,Female,1980.0,2004 Summer Olympics,Athletics,"Shot Put, Women",Olympic,NaN,3,NaN,Belarus,2004,1,24.0,0,0
222155,Vadim Devyatovsky,Male,1977.0,2004 Summer Olympics,Athletics,"Hammer Throw, Men",Olympic,NaN,3,NaN,Belarus,2004,1,27.0,1,0
223700,Goldie Sayers,Female,1982.0,2008 Summer Olympics,Athletics,"Javelin Throw, Women",Olympic,NaN,3,NaN,Great Britain,2008,1,26.0,0,0
224842,Ruth Beitía,Female,1979.0,2012 Summer Olympics,Athletics,"High Jump, Women",Olympic,NaN,3,NaN,Spain,2012,1,33.0,0,0
241189,Chelsea Hammond,Female,1983.0,2008 Summer Olympics,Athletics,"Long Jump, Women",Olympic,NaN,3,NaN,Jamaica,2008,1,25.0,0,0
251912,Abeba Aregawi,Female,1990.0,2012 Summer Olympics,Athletics,"1,500 metres, Women",Olympic,NaN,3,NaN,Ethiopia,2012,1,22.0,0,0
253584,Ivan Yefremov,Male,1986.0,2012 Summer Olympics,Weightlifting,"Heavyweight, Men",Olympic,NaN,3,NaN,Uzbekistan,2012,1,26.0,1,0
260145,Alfonso Benavides,Male,1991.0,2012 Summer Olympics,Canoe Sprint (Canoeing),"Canadian Singles, 200 metres, Men",Olympic,NaN,3,NaN,Spain,2012,1,21.0,1,0


In [13]:
# Blessing Okagbare promoted to Silver in 2017
df.loc[(df.Name == "Blessing Okagbare") & (df.Games_Year == 2008) & (df.Event == "Long Jump, Women"), "Medal"] = "Silver"
# Chelsea Hammond promoted to Bronze in 2017
df.loc[(df.Name == "Chelsea Hammond") & (df.Games_Year == 2008) & (df.Event == "Long Jump, Women"), "Medal"] = "Bronze"
# Create numerical Position variable
num_pos = []
for p in df["Pos"]:
    try:
        n = int(p.replace("=",""))
        num_pos.append(n)
    except:
        num_pos.append(np.nan)
df["Num_Pos"] = num_pos

In [14]:
# Standardize Nationality
from lxml import html
import requests
url = "http://www.olympedia.org/countries/"
response = requests.get(url)
tree = html.document_fromstring(response.text)
nocs = []
nats = []
rows = tree.xpath("//table[@class='table table-striped sortable'][1]//tbody/tr")
for r in rows:
    entries = r.xpath("td")
    assert len(entries) == 3
    nocs.append(" ".join(entries[0].text_content().split()))
    nats.append(" ".join(entries[1].text_content().split()))
# Account for modern or alternative names
nocs.remove("SWZ")
nats.remove("Eswatini")
nocs.append("HKG")
nats.append("Hong Kong")
nocs.append("CHN")
nats.append("China")
nocs.append("RUS")
nats.append("Russia")
nocs.append("IRI")
nats.append("Iran")
nocs.append("SWZ")
nats.append("Swaziland")
nocs.append("MKD")
nats.append("Former Yugoslav Republic of Macedonia")
nocs.append("IOA")
nats.append("Independent Olympic Participants")
nocs.append("IOA")
nats.append("Independent Olympic Athletes")
# Fix specific problematic strings
df.loc[df.Nat == "Sweden as Malin Baryard", "Nat"] = "Sweden"
df.loc[df.Nat == "Italy as Francesca Bortolozzi", "Nat"] = "Italy"
df.loc[df.Nat == "Germany as Olga Jordan", "Nat"] = "Germany"
df.loc[df.Nat == "United States as Jordan Larson", "Nat"] = "United States"
# Create Country and NOC columns
nats_col = []
nocs_col = []
for entry in df.Nat:
    e_nat = ""
    e_noc = np.nan
    for n in nats:
        if n in entry:
            if len(n) > len(e_nat):
                # The longer match is better
                e_nat = n
                e_noc = nocs[nats.index(n)]            
    if len(e_nat) == 0:
        e_nat = entry
    nats_col.append(e_nat)
    nocs_col.append(e_noc)
# Assign NOCs with multiple country name strings to just one string
for i in range(len(nocs_col)):
    if nocs_col[i] in nocs:
        nats_col[i] = nats[nocs.index(nocs_col[i])]
df["Country"] = nats_col
df["NOC"] = nocs_col

In [15]:
# These are outdated Country names, which are not assigned NOCs
df.loc[~df.Country.isin(set(nats))].Country.unique()

array(['Dahomey', 'Cape Colony', 'British Guiana', 'Burma', 'Zaire',
       'Ceylon', 'Taiwan', 'Formosa', 'Eswatini', 'Syria', 'Upper Volta',
       'Gold Coast', 'Tanganyika', 'Ottoman Empire'], dtype=object)

In [16]:
# Assign host nation
df["Host"] = np.nan
df.loc[(df.Games == "1896 Summer Olympics"), "Host"] = "Greece"
df.loc[(df.Games == "1900 Summer Olympics"), "Host"] = "France"
df.loc[(df.Games == "1904 Summer Olympics"), "Host"] = "United States"
df.loc[(df.Games == "1908 Summer Olympics"), "Host"] = "Great Britain"
df.loc[(df.Games == "1912 Summer Olympics"), "Host"] = "Sweden"
df.loc[(df.Games == "1920 Summer Olympics"), "Host"] = "Belgium"
df.loc[(df.Games == "1924 Summer Olympics"), "Host"] = "France"
df.loc[(df.Games == "1924 Winter Olympics"), "Host"] = "France"
df.loc[(df.Games == "1928 Summer Olympics"), "Host"] = "Netherlands"
df.loc[(df.Games == "1928 Winter Olympics"), "Host"] = "Switzerland"
df.loc[(df.Games == "1932 Summer Olympics"), "Host"] = "United States"
df.loc[(df.Games == "1932 Winter Olympics"), "Host"] = "United States"
df.loc[(df.Games == "1936 Summer Olympics"), "Host"] = "Germany"
df.loc[(df.Games == "1936 Winter Olympics"), "Host"] = "Germany"
df.loc[(df.Games == "1948 Summer Olympics"), "Host"] = "Great Britain"
df.loc[(df.Games == "1948 Winter Olympics"), "Host"] = "Switzerland"
df.loc[(df.Games == "1952 Summer Olympics"), "Host"] = "Finland"
df.loc[(df.Games == "1952 Winter Olympics"), "Host"] = "Norway"
df.loc[(df.Games == "1956 Equestrian Olympics"), "Host"] = "Sweden"
df.loc[(df.Games == "1956 Summer Olympics"), "Host"] = "Australia"
df.loc[(df.Games == "1956 Winter Olympics"), "Host"] = "Italy"
df.loc[(df.Games == "1960 Summer Olympics"), "Host"] = "Italy"
df.loc[(df.Games == "1960 Winter Olympics"), "Host"] = "United States"
df.loc[(df.Games == "1964 Summer Olympics"), "Host"] = "Japan"
df.loc[(df.Games == "1964 Winter Olympics"), "Host"] = "Austria"
df.loc[(df.Games == "1968 Summer Olympics"), "Host"] = "Mexico"
df.loc[(df.Games == "1968 Winter Olympics"), "Host"] = "France"
df.loc[(df.Games == "1972 Summer Olympics"), "Host"] = "West Germany"
df.loc[(df.Games == "1972 Winter Olympics"), "Host"] = "Japan"
df.loc[(df.Games == "1976 Summer Olympics"), "Host"] = "Canada"
df.loc[(df.Games == "1976 Winter Olympics"), "Host"] = "Austria"
df.loc[(df.Games == "1980 Summer Olympics"), "Host"] = "Soviet Union"
df.loc[(df.Games == "1980 Winter Olympics"), "Host"] = "United States"
df.loc[(df.Games == "1984 Summer Olympics"), "Host"] = "United States"
df.loc[(df.Games == "1984 Winter Olympics"), "Host"] = "Yugoslavia"
df.loc[(df.Games == "1988 Summer Olympics"), "Host"] = "Republic of Korea"
df.loc[(df.Games == "1988 Winter Olympics"), "Host"] = "Canada"
df.loc[(df.Games == "1992 Summer Olympics"), "Host"] = "Spain"
df.loc[(df.Games == "1992 Winter Olympics"), "Host"] = "France"
df.loc[(df.Games == "1994 Winter Olympics"), "Host"] = "Norway"
df.loc[(df.Games == "1996 Summer Olympics"), "Host"] = "United States"
df.loc[(df.Games == "1998 Winter Olympics"), "Host"] = "Japan"
df.loc[(df.Games == "2000 Summer Olympics"), "Host"] = "Australia"
df.loc[(df.Games == "2002 Winter Olympics"), "Host"] = "United States"
df.loc[(df.Games == "2004 Summer Olympics"), "Host"] = "Greece"
df.loc[(df.Games == "2006 Winter Olympics"), "Host"] = "Italy"
df.loc[(df.Games == "2008 Summer Olympics"), "Host"] = "People's Republic of China"
df.loc[(df.Games == "2010 Winter Olympics"), "Host"] = "Canada"
df.loc[(df.Games == "2012 Summer Olympics"), "Host"] = "Great Britain"
df.loc[(df.Games == "2014 Winter Olympics"), "Host"] = "Russian Federation"
df.loc[(df.Games == "2016 Summer Olympics"), "Host"] = "Brazil"
df.loc[(df.Games == "2018 Winter Olympics"), "Host"] = "Republic of Korea"

In [17]:
# Assign games order
df["Games_Order"] = np.nan
df.loc[(df.Games == "1896 Summer Olympics"), "Games_Order"] = 1
df.loc[(df.Games == "1900 Summer Olympics"), "Games_Order"] = 2
df.loc[(df.Games == "1904 Summer Olympics"), "Games_Order"] = 3
df.loc[(df.Games == "1908 Summer Olympics"), "Games_Order"] = 4
df.loc[(df.Games == "1912 Summer Olympics"), "Games_Order"] = 5
df.loc[(df.Games == "1920 Summer Olympics"), "Games_Order"] = 6
df.loc[(df.Games == "1924 Summer Olympics"), "Games_Order"] = 8
df.loc[(df.Games == "1924 Winter Olympics"), "Games_Order"] = 7
df.loc[(df.Games == "1928 Summer Olympics"), "Games_Order"] = 10
df.loc[(df.Games == "1928 Winter Olympics"), "Games_Order"] = 9
df.loc[(df.Games == "1932 Summer Olympics"), "Games_Order"] = 12
df.loc[(df.Games == "1932 Winter Olympics"), "Games_Order"] = 11
df.loc[(df.Games == "1936 Summer Olympics"), "Games_Order"] = 14
df.loc[(df.Games == "1936 Winter Olympics"), "Games_Order"] = 13
df.loc[(df.Games == "1948 Summer Olympics"), "Games_Order"] = 16
df.loc[(df.Games == "1948 Winter Olympics"), "Games_Order"] = 15
df.loc[(df.Games == "1952 Summer Olympics"), "Games_Order"] = 18
df.loc[(df.Games == "1952 Winter Olympics"), "Games_Order"] = 17
df.loc[(df.Games == "1956 Equestrian Olympics"), "Games_Order"] = 20
df.loc[(df.Games == "1956 Summer Olympics"), "Games_Order"] = 20
df.loc[(df.Games == "1956 Winter Olympics"), "Games_Order"] = 19
df.loc[(df.Games == "1960 Summer Olympics"), "Games_Order"] = 22
df.loc[(df.Games == "1960 Winter Olympics"), "Games_Order"] = 21
df.loc[(df.Games == "1964 Summer Olympics"), "Games_Order"] = 24
df.loc[(df.Games == "1964 Winter Olympics"), "Games_Order"] = 23
df.loc[(df.Games == "1968 Summer Olympics"), "Games_Order"] = 26
df.loc[(df.Games == "1968 Winter Olympics"), "Games_Order"] = 25
df.loc[(df.Games == "1972 Summer Olympics"), "Games_Order"] = 28
df.loc[(df.Games == "1972 Winter Olympics"), "Games_Order"] = 27
df.loc[(df.Games == "1976 Summer Olympics"), "Games_Order"] = 30
df.loc[(df.Games == "1976 Winter Olympics"), "Games_Order"] = 29
df.loc[(df.Games == "1980 Summer Olympics"), "Games_Order"] = 32
df.loc[(df.Games == "1980 Winter Olympics"), "Games_Order"] = 31
df.loc[(df.Games == "1984 Summer Olympics"), "Games_Order"] = 34
df.loc[(df.Games == "1984 Winter Olympics"), "Games_Order"] = 33
df.loc[(df.Games == "1988 Summer Olympics"), "Games_Order"] = 36
df.loc[(df.Games == "1988 Winter Olympics"), "Games_Order"] = 35
df.loc[(df.Games == "1992 Summer Olympics"), "Games_Order"] = 38
df.loc[(df.Games == "1992 Winter Olympics"), "Games_Order"] = 37
df.loc[(df.Games == "1994 Winter Olympics"), "Games_Order"] = 39
df.loc[(df.Games == "1996 Summer Olympics"), "Games_Order"] = 40
df.loc[(df.Games == "1998 Winter Olympics"), "Games_Order"] = 41
df.loc[(df.Games == "2000 Summer Olympics"), "Games_Order"] = 42
df.loc[(df.Games == "2002 Winter Olympics"), "Games_Order"] = 43
df.loc[(df.Games == "2004 Summer Olympics"), "Games_Order"] = 44
df.loc[(df.Games == "2006 Winter Olympics"), "Games_Order"] = 45
df.loc[(df.Games == "2008 Summer Olympics"), "Games_Order"] = 46
df.loc[(df.Games == "2010 Winter Olympics"), "Games_Order"] = 47
df.loc[(df.Games == "2012 Summer Olympics"), "Games_Order"] = 48
df.loc[(df.Games == "2014 Winter Olympics"), "Games_Order"] = 49
df.loc[(df.Games == "2016 Summer Olympics"), "Games_Order"] = 50
df.loc[(df.Games == "2018 Winter Olympics"), "Games_Order"] = 51

In [18]:
# Assign is_at_home for athletes
df_hc_tups = zip(df["Host"], df["Country"])
is_at_home = []
for t in df_hc_tups:
    if t[0] == t[1]:
        is_at_home.append(1)
    else:
        is_at_home.append(0)
df["is_at_home"] = is_at_home

In [19]:
# Write cleaned
df = df[["Name", "Games", "Games_Order", "Games_Year", "is_summer", "Host", "Sport", "Event", "Num_Pos", "Medal", "is_team_event", "Team", "Male", "Birth", "Age", "NOC", "Country", "is_at_home"]]
df.to_csv("../data/athletes_clean.txt", index = False, sep = "\t")